<a href="https://colab.research.google.com/github/Marilu27/AR_BCI_ROBOT_Moverio/blob/master/ARBCIROBOT_intrasoggettivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install mne

     |████████████████████████████████| 6.9MB 4.2MB/s 


In [29]:
!pwd utilsForDataset
import sys 

/content


In [33]:
import gc
import sys 
sys.path.append('/content/ar_bci_robot')
#from ar_bci_robot import*
#from ar_bci_robot import run
from run import run
import numpy as np
from ar_bci_robot import dataFuns 
from dataFuns import Percent 
import pandas as pd



if __name__ == '__main__':
    gc.collect()


    ''' setting intra-soggettivo   '''
    selSubJectsArr = ['CanAle', 'CapFra', 'CicMel', 'CioAnt', 'CraFed', 'CraSim', 'DasCre', 'DeAGio', 'DeLAnn', 'ErrErn', 'EspAnt', 'FalGia', 'FroMir', 'MocNic', 'PasLor', 'PesMar', 'PetPas', 'SpeMar', 'TeoAle', 'VasBen']
    epochInSecondsArr = [3]
    overlapInSecondsArr = [0] # # massimo 1 secondo, 0.5 o 1 s # con 10 in epoch mettere 0 per prima..
    filterButterwothArr = [False] #
    folds_arr = [1] # 1 per split train test per soggetto,
    #folds_arr = [10] # 10 per k-fold, con k=10


    ''' setting inter-soggettivo   '''
    # selSubJectsArr = ['ALL']
    # epochInSecondsArr = [3]
    # overlapInSecondsArr = [0] # con 10 in epoch mettere 0 per prima..
    # filterButterwothArr = [False]
    # folds_arr = [20] # LOSO
    # #folds_arr = [10] # K-fold


    epochXoverlap = 1
    numberOfSettings = len(selSubJectsArr) * epochXoverlap * len(filterButterwothArr) * len(folds_arr)
    saveFiles = True # todo sistemare per 10-fold..

    # combina i setting
    n = 1
    for epoch in epochInSecondsArr:
        for overlap in overlapInSecondsArr:
            if epoch == 10 and overlap != 0:  # con epoch 10 è inutile l'overlap
                break
            for filterB in filterButterwothArr:
                for folds in folds_arr:

                    acc = {}  # accuracy per soggetto e modello
                    for subject in selSubJectsArr:

                        settingStr = ' epoch: ' + str(epoch) + ' overlap: ' + str(overlap) + ' filterButterwoth: ' + str(filterB) + ' folds: ' + str(folds)

                        print(f'\n\n\n\n#### setting:', n, '/', numberOfSettings, 'subject:', subject, settingStr, ' ####')

                        # esegue un run su un soggetto, un setting settato, su modelli knn, svm, net
                        acc[subject] = run(epoch, overlap, subject, filterB, saveFiles, folds)

                        n += 1

                    if saveFiles:
                        if subject != 'ALL' and folds >= 1: # todo controllare bene se è ok...
                            # calcola le medie e le std per soggetto/modello
                            #
                            #
                            models = acc['CanAle'].keys()
                            subjects = acc.keys()
                            meansAcc = {}
                            stdsAcc = {}

                            for m in models:
                                acc_subjects = []

                                for s in subjects:
                                    acc_subjects.append(acc[s][m])

                                acc_subjects = np.array(acc_subjects).astype(np.float32)
                                meansAcc[m] = Percent(np.mean(acc_subjects)).__str__()
                                stdsAcc[m] = Percent(np.std(acc_subjects)).__str__()


                            # stampa di riepilogo/check
                            #
                            #
                            print('\n\n---------------------------------------------------------------------\nSETTING: ' + settingStr)

                            print('\nACCURACY: ')
                            for subj in acc:
                                print('\t' + subj)
                                for model in acc[subj]:
                                    print(
                                        '\t\t' + model + '\t' + Percent(acc[subj][model]).__str__())
                            print('\t-MEAN / STD:')
                            for m in models:
                                print('\t\t' + m + ': ' + meansAcc[m] + ' / ' + stdsAcc[m])


                            # salva su file le medie e le std
                            #
                            #
                            for m in models:
                                d = {
                                    'subject': 'mean±std',
                                    # accuracy
                                    'acc': 'acc ' + meansAcc[m] + '±' + stdsAcc[m],
                                    # setting file name
                                    'setting': m + ' - ' + settingStr,

                                }
                                df = pd.DataFrame(data=d, index=[0])
                                df.to_csv('run_MeanStd.csv', mode='a', header=False, index=False, sep='\t')









#### setting: 1 / 20 subject: CanAle  epoch: 3 overlap: 0 filterButterwoth: False folds: 1  ####


NameError: ignored